In [53]:
import re
all_punctuation = '!,;:?"\'、，；.()<>&$\/\`[\]\-\—+£'
sub_punctuation = '[.,?!]'
def removePunctuation(text):
    text = re.sub(r'[{}]+'.format(sub_punctuation),' ',text)
    return text.strip()


In [2]:
# 不要重复执行
with open("D:\\MUST\\2020\\11.14\\MRPC\\train.tsv",'r',encoding='utf-8') as f:
    lines = f.readlines()
 
text = []
for line in lines[1:]:
    label = line.split("\t")[0]
    txt_before = line.split("\t")[3]
    txt_after = line.split("\t")[4]
    text.append([label,txt_before,txt_after])
text
with open("D:\\MUST\\2021\\MRPCtrain.txt",'w',encoding='utf-8') as f:
    for line in text:
        f.write("\t".join(line))

In [54]:
with open("D:\\MUST\\2021\\MRPCtrain.txt",'r',encoding='utf-8') as f:
    lines = f.readlines() 

In [66]:
#数据集大小
dataset_size = len(lines)

In [67]:
label_list = []
beforetxt_list = []
aftertxt_list = []
for line in lines[:dataset_size]:
    line = line.split('\t')
    label_list.append(line[0])
    beforetxt_list.append(line[1])
    aftertxt_list.append(line[2].replace("\n",""))

In [68]:
import re
import math
import torch
import numpy as np
from random import *
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

In [69]:
alltxt_list = beforetxt_list+aftertxt_list
alltxt = " ".join(alltxt_list)
#去掉所有句子的标点符号
sentences = re.sub("[\"$\'\’\-\(\)%.,?!]",'',alltxt.lower()).split('\n')
#所有词的列表
word_list = list(set(" ".join(sentences).split()))
#生成带索引的词字典，并添加上四种标志词
word2idx = {'[PAD]':0,'[CLS]':1,'[SEP]':2,'[MASK]':3}
for i,w in enumerate(word_list):
    word2idx[w] = i+4
# 对换word2idx的key和value->idx2word
idx2word = dict(zip(word2idx.values(),word2idx.keys()))
# 上面的两个字典的大小
vocab_size = len(idx2word)
# token是所有句子的idx表示，每个句子一个list，所有句子组成一个大list
token_list = list()
for sentence in alltxt_list:
    sentence = re.sub('[\"$\'\’\-\(\)%.,?!]','',sentence.lower())
    idx_sentence = [word2idx[s] for s in sentence.split()]
    token_list.append(idx_sentence)

In [71]:
# BERT Parameters

# maxlen表示同一个batch中的所有句子都由30个token组成，不够的补PAD
# （这里我实现的方式比较粗暴，直接固定所有batch中的所有句子都为30）
maxlen = 100
batch_size = 6
# max_pred表示最多需要预测多少个单词，即BERT中的完形填空任务
max_pred = 8 
# n_layers表示Encoder Layer的数量
n_layers = 6
n_heads = 12
# d_model表示Token Embeddings、Segment Embeddings、Position Embeddings的维度
d_model = 768
# d_ff表示Encoder Layer中全连接层的维度
d_ff = 768*4 
d_k = d_v = 64  # dimension of K(=Q), V
# n_segments表示Decoder input由几句话组成
n_segments = 2



In [104]:
def make_data():
    i = 0
    batchs = []
    positive = negative = 0
    while positive!=batch_size/2 or negative!=batch_size/2:
        print("[",i,"]")
        i+=1
        #从sentences中随机取两句话的序号
        tokens_a_index = randrange(dataset_size)
        tokens_b_index = tokens_a_index+dataset_size
        print(tokens_a_index,tokens_b_index,int(label_list[tokens_a_index]))
        #取对应的句子的token（idx_list）
        tokens_a,tokens_b = token_list[tokens_a_index],token_list[tokens_b_index]
        #将两个句子合并，并且加上开头结尾符号和中间分隔符，得到一个list，input_tokens
        input_tokens = [word2idx['[CLS]']]+tokens_a+[word2idx['[SEP]']]+tokens_b+[word2idx['[SEP]']]
        #segment_tokens表示句子前后顺序
        segment_tokens = [0]*(1+len(tokens_a)+1)+[1]*(len(tokens_b)+1)
        #n_pred(编码器需要预测的也就是被遮盖住的词的个数)按原文要求取单个训练例子（两个句子结合的句子）长度的15%遮盖住，限定必须小于max_pred
        #n_pred = min(max_pred,len(input_index)*0.15)
        n_pred = int(len(input_tokens)*0.15)

        #指代了真实单词的位置，也就是可以被mask的单词的位置
        cand_masked_pos = [i for i,token in enumerate(input_tokens)
                          if token!=word2idx['[CLS]'] and token!=word2idx['[SEP]']]
        
        #将这个句子的所有单词的位置都打散
        shuffle(cand_masked_pos)
        #被遮住的词和被遮住的词的位置
        masked_tokens,masked_pos = [],[]
        #注：masked_tokens是原词，input_tokens中对应位置的是替换后的词
        #取前n_pred个词，因为已经打散了，这就相当于随机去了n_pred个
        for pos in cand_masked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_tokens[pos])
            if random()<0.8:
                input_tokens[pos] = word2idx['[MASK]']
            elif random()<0.1:
                input_tokens[pos] = randint(4,vocab_size-1)
        print(len(input_tokens))
        print(len(segment_tokens))
        #总句子不够长度的位置补[PAD]
        if len(input_tokens)<maxlen:
            input_tokens.extend([0]*(maxlen-len(input_tokens)))
            segment_tokens.extend([0]*(maxlen-len(segment_tokens)))
            #前面有写到n_pred<max_pred
            #E.g. input_ids = [1,8, 36, 27, 13, 39, 33, 34,2,39, 33, 35, 26, 30, 38, 17, 5, 22, 16, 6, 12,2]
            #     (shuffle)cand_maked_pos = [4, 17, 13, 5, 6, 14, 2, 15, 18, 20, 1, 3, 19, 7, 12, 16, 11, 9, 10]
            #     len:19  ;  maxlen:30  ;   n_pred = 19*0.15 = 2.85 = 2  ;  max_pred = 5
            #     masked_pos:[4,17]  masked_tokens:[13,22]
            #     (masked)input_ids:[4->4, 17->4, 13, 5, 6, 14, 2, 15, 18, 20, 1, 3, 19, 7, 12, 16, 11, 9, 10]
            #     masked_pos(补零):[4, 17, 0, 0, 0] masked_tokens(补零):[13,22,0,0,0]
            masked_tokens.extend([0]*(max_pred-n_pred))
            masked_pos.extend([0]*(max_pred-n_pred))
        print(len(input_tokens))
        print(len(segment_tokens))
        #tokens_a_index + 3668 == tokens_b_index且label_list[tokens_a_index]==1
#         :表示两个句子相邻，是上下文关系
        #positive：表示随机抽到的句子对中，两个句子相邻，的次数
        if label_list[tokens_a_index]=='1' and positive < batch_size/2:
            print("positive")
            print(label_list[tokens_a_index])
            batchs.append([input_tokens, segment_tokens, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif label_list[tokens_a_index]=='0' and negative < batch_size/2:
            print("negative")
            print(label_list[tokens_a_index])
            batchs.append([input_tokens, segment_tokens, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
      
    print(i,positive,negative)
    return batchs    

In [109]:
batchs = make_data()

[ 0 ]
973 4641 0
35
35
100
100
negative
0
[ 1 ]
1236 4904 1
54
54
100
100
positive
1
[ 2 ]
2972 6640 0
27
27
100
100
negative
0
[ 3 ]
2814 6482 0
51
51
100
100
negative
0
[ 4 ]
1431 5099 1
55
55
100
100
positive
1
[ 5 ]
3404 7072 0
35
35
100
100
[ 6 ]
851 4519 1
35
35
100
100
positive
1
7 3 3


In [106]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = zip(*batchs)
#转换为torch张量
input_ids, segment_ids, masked_tokens, masked_pos, isNext = \
    torch.LongTensor(input_ids),  torch.LongTensor(segment_ids), torch.LongTensor(masked_tokens),\
    torch.LongTensor(masked_pos), torch.LongTensor(isNext)

In [112]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext

(tensor([[    1,     3,   636,  8008,  2549,     3,  5452, 11061,  5178, 12574,
           2359,  5993,     2,  3464,     3,  1575,   636,  4992,  4891, 11061,
           5178,  5452,  2148,     3, 12574,  2359,  5993,     2,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [    1,     3,  2184,  2757,  7028,     3,  1873, 11726,  5155, 12401,
           6343,  4367,  6343, 13374, 10295,  6798,  9159,  2713, 11061,     3,
              2,  4291,     3,  7795,  

In [108]:
class MyDataSet(Data.Dataset):
  def __init__(self, input_ids, segment_ids, masked_tokens, masked_pos, isNext):
    self.input_ids = input_ids
    self.segment_ids = segment_ids
    self.masked_tokens = masked_tokens
    self.masked_pos = masked_pos
    self.isNext = isNext
  
  def __len__(self):
    return len(self.input_ids)
  
  def __getitem__(self, idx):
    return self.input_ids[idx], self.segment_ids[idx], self.masked_tokens[idx], self.masked_pos[idx], self.isNext[idx]

loader = Data.DataLoader(MyDataSet(input_ids, segment_ids, masked_tokens, masked_pos, isNext), batch_size, True)

数据预处理部分结束

In [120]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding,self).__init__()
        self.tok_embed = nn.Embedding(vocab_size,d_model)#对总词典进行embedding编码
        self.pos_embed = nn.Embedding(maxlen,d_model)#对每个位置进行embedding编码
        self.seg_embed = nn.Embedding(n_segments,d_model)#对segment的值0/1进行embedding编码
        self.norm = nn.LayerNorm(d_model)
        
    def forward(self,input_tokens,input_segments):
        #input_x：是一个batch=6个句子对，每个句子对的所有词的idx组成一个list，即二维
        #input_pos：每个句子对的位置索引list，没有传入，在下面这三行生成
        #下面这三行相当于把pos(input)扩展成和input_x,input_segment一样的shape
        #1.第二维大小=maxlen=100
        seq_len = maxlen
        #2.取这个batch中所有位置索引[0,1,2...,maxlen-1]
        seq_pos = torch.arange(seq_len,dtype=torch.long)
        #seq_pos = torch.arange(seq_len,dtype=torch.int8)
        
        #3.unsqueeze在第一个维度前增加一个维度，E.g.shape[100]->shape[1,100]
        #expand_as将seq_pos扩展成和input_tokens一样的shape[6,100]
        seq_pos = seq_pos.unsqueeze(0).expand_as(input_tokens)
        
        
        #调用自带的embedding函数，分别对input_tokens,seq_pos,input_segments生成三个embed值，累加
        embedding = self.tok_embed(input_tokens)+self.pos_embed(seq_pos)+self.seg_embed(input_segments)
        #归一化
        return self.norm(embedding)
        

In [122]:
embedding = Embedding()
tok_embed = nn.Embedding(vocab_size,d_model)

i = 0
for input_tokens, segment_tokens, masked_tokens, masked_pos, isNext in loader:
    if i==1:
        break
    print("input_ids:",input_tokens.size())
    print("\nsegment_ids:",segment_tokens.size())
    print("\nmasked_tokens:",masked_tokens.size())
    print("\nmasked_pos:",masked_pos.size())
    print("\nisNext:",isNext.size())
    i+=1
    t_e = tok_embed(input_tokens)
    print("\nt_e_size:",t_e.size())
    print("\nt_e:",t_e)
    output = embedding(input_tokens,segment_tokens)
    print("\noutput_size:",output.size())
    print("\noutput:",output)

input_ids: torch.Size([6, 100])

segment_ids: torch.Size([6, 100])

masked_tokens: torch.Size([6, 8])

masked_pos: torch.Size([6, 8])

isNext: torch.Size([6])

t_e_size: torch.Size([6, 100, 768])

t_e: tensor([[[-1.6172,  0.3280,  0.9971,  ...,  0.0850, -0.1743,  0.6148],
         [ 0.5803,  1.9750,  0.5276,  ..., -0.8609, -0.2403, -0.3550],
         [ 0.6973, -0.5578, -0.0442,  ...,  0.2790, -1.1133,  0.7884],
         ...,
         [ 0.5494, -0.1238,  1.7511,  ...,  0.4101, -0.5980, -2.2431],
         [ 0.5494, -0.1238,  1.7511,  ...,  0.4101, -0.5980, -2.2431],
         [ 0.5494, -0.1238,  1.7511,  ...,  0.4101, -0.5980, -2.2431]],

        [[-1.6172,  0.3280,  0.9971,  ...,  0.0850, -0.1743,  0.6148],
         [-1.2548,  0.1464, -0.4946,  ...,  0.1805, -1.2416,  0.2647],
         [ 0.5803,  1.9750,  0.5276,  ..., -0.8609, -0.2403, -0.3550],
         ...,
         [ 0.5494, -0.1238,  1.7511,  ...,  0.4101, -0.5980, -2.2431],
         [ 0.5494, -0.1238,  1.7511,  ...,  0.4101, -0.598

In [ ]:
def get_attn_pad_mask(input_tokens_q,input_tokens_k):
    batch_size,seq_len = input_tokens_q.size()
    pad_attn_mask = input_tokens_q.data.eq(0).unsqueeze(1)
    return pad_attn_mask.expand(batch_size,seq_len,seq_len)#[6,100,100]全是True/False

def gelu(x):
    return x*0.5*(1.0+torch.erf(x/math.sqrt(2.0)))

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention,self).__init__()
        
    def forward(self,Q,K,V,attn_mask):
        scores = torch.matmul(Q,K.transpose(-1,-2))/np.sqrt(d_k)
        scores.masked_fill_(attn_mask,-1e9)# Fills elements of self tensor with value where mask is 1或者True.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn,V)
        return context

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention,self).__init__()
        self.L_Q = nn.Linear(d_model,d_k*n_heads)
        self.L_K = nn.Linear(d_model,d_k*n_heads)
        self.L_V = nn.Linear(d_model,d_v*n_heads)
        
    def forward(self,Q,K,V,attn_mask):
        #[6, 100, 12, 64]-->[6, 12, 100, 64]
        q_s = self.L_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size, n_heads, seq_len, d_k]
        k_s = self.L_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size, n_heads, seq_len, d_k]
        v_s = self.L_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size, n_heads, seq_len, d_v]
        
        #attn_mask.unsqueeze(1).repeat(1,12,1,1)==attn_mask.unsqueeze(1).expand(6,12,100,100)
        attn_mask = attn_mask.unsqueeze(1).repeat(1,n_heads,1,1)
        #attn_mask = attn_mask.unsqueeze(1).expand(6,12,100,100)
        
        context = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        #####原Transformer论文中的CONCAT????是这里吗？
        context = context.transpose(1,2).contiguous().view(batch_size,-1, n_heads * d_v)
        #####原Transformer论文中的Linear
        output = nn.Linear(n_heads * d_v, d_model)(context)
        #####原Transformer论文中的每一个Multiheadattention后都跟了一个layernorm
        return nn.LayerNorm(d_model)(output + residual) # output: [batch_size, seq_len, d_model]


In [ ]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet,self).__init__()
        self.fc1 = nn.Linear(d_model,d_ff)
        self.fc2 = nn.Linear(d_ff,d_model)
        
    def forward(self,x):
        return self.fc2(gelu(self.fc1(x)))

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer,self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()
        
    def forward(self,embedded_input_tokens,pad_attn_mask):
        enc_outputs = self.enc_self_attn(embedded_input_tokens,embedded_input_tokens,embedded_input_tokens,pad_attn_mask)
        enc_outputs = self.pos_ffn(enc_outputs)
        return enc_outputs        

In [123]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT,self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])#创建n_layers个encoderlayer层
#         未完！！！！！！！   
        self.fc = nn.Sequential(
            nn.Linear(d_model,d_model),
            nn.Dropout(0.5),
            nn.Tanh(),
        )
        self.classifier = nn.Linear(d_model,2)
        self.linear = nn.Linear(d_model,d_model)
        self.activ2 = gelu
        embed_weight = self.embedding.tok_embed.weight
        self.fc2 = nn.Linear(d_model,vocab_size,bias=False)
        self.fc2.weight = embed_weight
        
    def forward(self,input_tokens,segment_tokens,masked_pos):
        embedded_input_tokens = self.embedding(input_tokens,segment_tokens)
        pad_attn_mask = get_attn_pad_mask(input_tokens,input_tokens)
        for layer in self.layers:
            embedded_input_tokens = layer(embedded_input_tokens,pad_attn_mask)
        enc_output = embedded_input_tokens
        
        h_pooled = self.fc(enc_output[:, 0]) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2] predict isNext

#         未完！！！！！！！
        masked_pos = masked_pos[:, :, None].expand(-1, -1, d_model) # [batch_size, max_pred, d_model]
        h_masked = torch.gather(enc_output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked = self.activ2(self.linear(h_masked)) # [batch_size, max_pred, d_model]
        logits_lm = self.fc2(h_masked) # [batch_size, max_pred, vocab_size]
        return logits_lm, logits_clsf
        
        
        

SyntaxError: unexpected EOF while parsing (<ipython-input-123-68ff90a654ec>, line 11)